We ran `2PV7` on our GPUs
[as above](https://github.com/google-deepmind/alphafold3/issues/59#issuecomment-2482720962)
with commit `ea04034` of the alphafold3 repository.
Ranking scores are now between 0.64 and 0.67, and the predicted structures generally align/match in pymol.

* We use the
[following script](https://github.com/jurgjn/alphafold_on_euler/blob/main/batch-infer/workflow/scripts/run_alphafold.sh)
to fix GPU capability 7.x, and enable unified memory + adjust shard spec for <40GB-ish GPU RAM
* Pre-Ampere GPUs (`rtx_2080_ti`, `titan_rtx`, `quadro_rtx_6000`, `v100`) only run with `--flash_attention_implementation=xla`
* Our `rtx_2080_ti` has 11GB GPU RAM, we are only able to run the example by forcing the compilation bucket to match the example size (`--buckets=596`). The predicted structure has a different orientation (predictions from all other GPUs are ordered identically), but does align to the A100 prediction (RMSD=4.2).

In [1]:
import os, os.path, json
import pandas as pd

In [2]:
gpus = [
    'alphafold3_predictions_rtx_2080_ti',
    'alphafold3_predictions_rtx_3090',
    'alphafold3_predictions_rtx_4090',
    'alphafold3_predictions_titan_rtx',
    'alphafold3_predictions_quadro_rtx_6000',
    'alphafold3_predictions_v100',
    'alphafold3_predictions_a100_pcie_40gb',
    'alphafold3_predictions_a100_80gb',
]

def score_(fp_):
    with open(fp_, 'r') as fh:
        js = json.load(fh)
        return js['ranking_score']

df_ = pd.DataFrame({'step': gpus})
df_['gpu'] = df_['step'].str.removeprefix('alphafold3_predictions_')
df_['json'] = df_['step'].map(lambda gpu: f'{gpu}/example_2pv7/example_2pv7_summary_confidences.json')
df_['isfile'] = df_['json'].map(os.path.isfile)
df_['ranking_score'] = df_['json'].map(score_)
df_[['gpu', 'ranking_score']]

,gpu,ranking_score
0,rtx_2080_ti,0.64
1,rtx_3090,0.67
2,rtx_4090,0.67
3,titan_rtx,0.67
4,quadro_rtx_6000,0.67
5,v100,0.66
6,a100_pcie_40gb,0.66
7,a100_80gb,0.66


In [3]:
df_[['gpu', 'ranking_score']].to_markdown('summary.md', index=False)